In [24]:
# Load all the GDX files
from collections import OrderedDict

import gdx
import pandas as pd
import xray

FILES = [
    ('bau', 'result_urban_exo.gdx'),
    ('3', 'result_cint_n_3.gdx'),
    ('4', 'result_cint_n_4.gdx'),
    ('5', 'result_cint_n_5.gdx'),    
    ]

raw = OrderedDict()
for case, fn in FILES:
    raw[case] = gdx.File('gdx/' + fn)

CREM = raw['bau']
cases = pd.Index(raw.keys(), name='case')
time = pd.Index(filter(lambda t: int(t) <= 2030, CREM.set('t')))

In [82]:
# List of all the parameters available in each file
#CREM.parameters()

In [99]:
arrays = {}

# GDP
temp = [raw[case].extract('gdp_ref') for case in cases]
arrays['GDP'] = xray.concat(temp, dim=cases).sel(rs=CREM.set('r')) \
                    .rename({'rs': 'r'})

In [100]:
# CO2 emissions
temp = []
for case in cases:
    temp.append(raw[case].extract('sectem').sum('g') +
        raw[case].extract('houem'))
arrays['CO2_emi'] = xray.concat(temp, dim=cases)

In [104]:
# Air pollutant emissions
temp = []
for case in cases:
    temp.append(raw[case].extract('urban').sum('*'))
temp = xray.concat(temp, dim=cases).sel(rs=CREM.set('r')).rename({'rs': 'r'})
for u in temp['urb']:
    if u in ['PM10', 'PM25']:
        continue
    arrays['{}_emi'.format(u.values)] = temp.sel(urb=u).drop('urb')

In [106]:
# Combine all variables into a single xray.Dataset; truncate time
data = xray.Dataset(arrays).sel(t=time)
data

<xray.Dataset>
Dimensions:  (case: 4, r: 30, t: 6)
Coordinates:
  * t        (t) <U4 '2007' '2010' '2015' '2020' '2025' '2030'
  * case     (case) object 'bau' '3' '4' '5'
  * r        (r) <U3 'BJ' 'TJ' 'HE' 'LN' 'SH' 'JS' 'ZJ' 'FJ' 'SD' 'GD' 'HI' ...
Data variables:
    GDP      (case, r, t) float64 127.6 155.0 221.5 310.5 403.3 503.6 70.65 ...
    OC_emi   (case, r, t) float64 0.02113 0.02014 0.01963 0.02251 0.0237 ...
    CO_emi   (case, r, t) float64 2.847 2.683 2.778 3.479 4.013 4.457 2.372 ...
    VOC_emi  (case, r, t) float64 0.934 0.9134 0.9477 1.241 1.478 1.675 ...
    CO2_emi  (case, r, t) float64 111.1 128.3 164.1 200.6 225.2 244.5 100.6 ...
    BC_emi   (case, r, t) float64 0.01766 0.01688 0.01721 0.01999 0.02121 ...
    NH3_emi  (case, r, t) float64 0.0755 0.06891 0.06917 0.08399 0.09497 ...
    SO2_emi  (case, r, t) float64 0.244 0.1953 0.2297 0.2647 0.2847 0.2973 ...
    NOX_emi  (case, r, t) float64 0.2941 0.2745 0.2745 0.3372 0.3823 0.417 ...

In [107]:
# Serialize to CSV: national totals
national = data.sum('r')

for c in cases:
    national.sel(case=c).drop('case').to_dataframe().to_csv('out/{}.csv'
                                                            .format(c))